In [1]:
import pandas as pd
import json

# Cargar el archivo CSV
df = pd.read_csv('/home/gmarco/workspace/canon_explicability/datasets/hanna/input/original_data/hanna-benchmark-asg-coling/hanna_stories_annotations.csv')

# Eliminar duplicados basados en las columnas 'Story ID', 'Model' y 'Story'
unique_stories = df.drop_duplicates(subset=['Story ID', 'Model', 'Story'])

# Crear una lista de diccionarios con el formato deseado
stories_list = []
for idx, row in unique_stories.iterrows():
    story_dict = {
        "story_idx": idx,
        "story_id": f"{row['Story ID']}_{row['Model']}",
        "story_name": row['Prompt'],
        "content": row['Story']
    }
    stories_list.append(story_dict)

# Guardar las historias en un archivo JSON
with open('unique_stories.json', 'w') as json_file:
    json.dump(stories_list, json_file, indent=4)

print("Historias únicas guardadas en unique_stories.json")

Historias únicas guardadas en unique_stories.json


In [2]:
import pandas as pd
from itertools import combinations
import math

# Cargar el archivo CSV
file_path = '/home/gmarco/workspace/canon_explicability/datasets/hanna/input/hanna_stories_annotations.csv'
df = pd.read_csv(file_path)

# Crear la columna 'story_id' combinando 'Story ID' y 'Model'
df['story_id'] = df['Story ID'].astype(str) + '_' + df['Model']

# Definir las columnas de puntuación a considerar
score_columns = ['Relevance', 'Coherence', 'Empathy', 'Surprise', 'Engagement', 'Complexity']

# Inicializar una lista para almacenar las comparaciones por pares
pairwise_data = []

# Generar comparaciones por pares considerando la suma de las puntuaciones
for worker_id, group in df.groupby('Worker ID'):
    num_texts = len(group)
    expected_pairs = math.comb(num_texts, 2)
    print(f"Worker ID: {worker_id}, Textos: {num_texts}, Pares esperados: {expected_pairs}")
    pairs_count = 0
    for (idx1, text1), (idx2, text2) in combinations(group.iterrows(), 2):
        # Calcular la suma de puntuación para cada historia
        score_text1 = text1[score_columns].sum()
        score_text2 = text2[score_columns].sum()
        
        # Comparar las puntuaciones y registrar la preferencia
        if score_text1 > score_text2:
            pairwise_data.append({
                'worker_id': worker_id,
                'preferred_text': text1['story_id'],
                'other_text': text2['story_id'],
                'label': 1
            })
            pairwise_data.append({
                'worker_id': worker_id,
                'preferred_text': text2['story_id'],
                'other_text': text1['story_id'],
                'label': 0
            })
        elif score_text1 < score_text2:
            pairwise_data.append({
                'worker_id': worker_id,
                'preferred_text': text2['story_id'],
                'other_text': text1['story_id'],
                'label': 1
            })
            pairwise_data.append({
                'worker_id': worker_id,
                'preferred_text': text1['story_id'],
                'other_text': text2['story_id'],
                'label': 0
            })
        
        pairs_count += 1
    
    # Comprobar si se generaron los pares esperados
    print(f"Worker ID: {worker_id}, Pares generados: {pairs_count}, Coincide con lo esperado: {pairs_count == expected_pairs}")

# Convertir los resultados a un DataFrame
pairwise_df = pd.DataFrame(pairwise_data)

# Convertir columnas a minúsculas
pairwise_df.columns = [col.lower() for col in pairwise_df.columns]

# Mostrar el resultado
print(pairwise_df)

# Exportar a CSV si es necesario
pairwise_df.to_csv('/home/gmarco/workspace/canon_explicability/datasets/hanna/output/pairwise_comparison_dataset.csv', index=False)

Worker ID: A03922113RU44GENR8ATX, Textos: 35, Pares esperados: 595
Worker ID: A03922113RU44GENR8ATX, Pares generados: 595, Coincide con lo esperado: True
Worker ID: A1IZ4NX41GKU4X, Textos: 942, Pares esperados: 443211
Worker ID: A1IZ4NX41GKU4X, Pares generados: 443211, Coincide con lo esperado: True
Worker ID: A1MJVTR0PCKBWW, Textos: 1, Pares esperados: 0
Worker ID: A1MJVTR0PCKBWW, Pares generados: 0, Coincide con lo esperado: True
Worker ID: A1SWRO4LMKPCOQ, Textos: 4, Pares esperados: 6
Worker ID: A1SWRO4LMKPCOQ, Pares generados: 6, Coincide con lo esperado: True
Worker ID: A1V6CP5I0TOSAR, Textos: 98, Pares esperados: 4753
Worker ID: A1V6CP5I0TOSAR, Pares generados: 4753, Coincide con lo esperado: True
Worker ID: A219VCQZADQ45W, Textos: 4, Pares esperados: 6
Worker ID: A219VCQZADQ45W, Pares generados: 6, Coincide con lo esperado: True
Worker ID: A23LXQ5ZX2YONR, Textos: 7, Pares esperados: 21
Worker ID: A23LXQ5ZX2YONR, Pares generados: 21, Coincide con lo esperado: True
Worker ID: A264

In [5]:
pairwise_df

,worker_id,preferred_text,other_text,label
0,A03922113RU44GENR8ATX,2_Human,8_Human,1
1,A03922113RU44GENR8ATX,8_Human,2_Human,0
2,A03922113RU44GENR8ATX,2_Human,29_Human,1
3,A03922113RU44GENR8ATX,29_Human,2_Human,0
4,A03922113RU44GENR8ATX,2_Human,39_Human,1
...,...,...,...,...
2047863,AYJGQBBK6KXZ7,807_Fusion,732_XLNet,0
2047864,AYJGQBBK6KXZ7,732_XLNet,999_TD-VAE,1
2047865,AYJGQBBK6KXZ7,999_TD-VAE,732_XLNet,0
2047866,AYJGQBBK6KXZ7,999_TD-VAE,807_Fusion,1


In [15]:
import pandas as pd
import json

# Supongamos que tu DataFrame se llama df
# Cargar el DataFrame desde un archivo CSV (si es necesario)
# df = pd.read_csv('path/to/your/csvfile.csv')

# Contar el número de comparaciones por cada worker_id

pairwise_df.rename(columns={'worker_id': 'user_id'}, inplace=True)

comparisons_count = pairwise_df['user_id'].value_counts()

# Filtrar los worker_id que tengan más de 100 comparaciones
filtered_worker_ids = comparisons_count[comparisons_count > 50].index

# Filtrar el DataFrame original para incluir solo los worker_id con más de 100 comparaciones
filtered_df = pairwise_df[pairwise_df['user_id'].isin(filtered_worker_ids)]


# Convertir el DataFrame filtrado a una lista de diccionarios
filtered_data = filtered_df.to_dict(orient='records')

# Guardar los datos filtrados en un archivo JSON
output_path = '/home/gmarco/workspace/canon_explicability/datasets_final/1_pairwise/hanna_pairs_balanced.json'
with open(output_path, 'w') as json_file:
    json.dump(filtered_data, json_file, indent=4)

print(f"Datos filtrados guardados en {output_path}")

Datos filtrados guardados en /home/gmarco/workspace/canon_explicability/datasets_final/1_pairwise/hanna_pairs_balanced.json


In [16]:
filtered_df.user_id.value_counts()

user_id
A1IZ4NX41GKU4X           813814
A2VE5IV9OD2SK1           716764
A3CFNUD7VR2E1E           408842
A2WNW8A4MOR7T7            61210
A264NN7JBX4UDQ            36112
A1V6CP5I0TOSAR             9080
A03922113RU44GENR8ATX      1116
AE861G0AY5RGT               668
AYJGQBBK6KXZ7               102
AHV4U78TUUDKI                66
Name: count, dtype: int64